In [ ]:
# def main_process(ranking_dict,distance_dict, voxel_size, aspect_ratio_threshold, cube_size, pcds_down_mapped):
#     combined_ratio_dict = {}
#     index = 1
#     # Iterate over each part in ranking_dict
#     for source_filename_stl, similar_parts_stl in ranking_dict.items():
        
#         # Replace the .stl extension with .pcd for the source filename
#         source_filename_pcd = source_filename_stl.replace('.stl', '.pcd')
#         source_pcd = pcds_down_mapped[source_filename_pcd]

#         aspect_ratio_source = calculate_aspect_ratio(source_pcd)
#         if aspect_ratio_source > aspect_ratio_threshold:
#             # Fetch top 50 tuples from the distance_dict for the given source file
#             if source_filename_stl in distance_dict:
#                 top_similar_parts_by_distance = distance_dict[source_filename_stl][:50]
#                 # Update combined_ratio_dict with these parts
#                 combined_ratio_dict[source_filename_pcd] = top_similar_parts_by_distance
#             continue
#         top_similar_parts_stl = similar_parts_stl[:50]  # Get top 50 similar parts    
#         for target_filename_stl in top_similar_parts_stl:
#             target_filename_pcd = target_filename_stl.replace('.stl', '.pcd')
#             target_pcd = pcds_down_mapped[target_filename_pcd]
#             aspect_ratio_target = calculate_aspect_ratio(target_pcd)
#             if aspect_ratio_target > aspect_ratio_threshold:
#                 update_metrics(combined_ratio_dict, source_filename_pcd, target_filename_pcd, 1)
#                 continue                
#             # Get deep copies for comparison and possible cropping
#             source_pcd_copy = copy.deepcopy(source_pcd)
#             target_pcd_copy = copy.deepcopy(target_pcd)

#             # Crop dimension if needed
#             # source_copy, target_copy,same_flag = crop_dimension_if_needed(source_extents, target_extents, source_pcd_copy, target_pcd_copy)
            
#             source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
#                 voxel_size, [source_pcd_copy, target_pcd_copy])
            
#             result_ransac = execute_global_registration(
#                 source_down, target_down, source_fpfh, target_fpfh, voxel_size, aspect_ratio_threshold)
            
#             # Assuming result_ransac and result_icp operations are successful
#             if result_ransac:
#                 source_down.transform(result_ransac.transformation)
#                 result_icp = refine_registration_with_icp(source_down, target_down, voxel_size)
#                 source_down.transform(result_icp.transformation)
#                 combined_ratio = calculate_and_visualize_subtracted_point_clouds(
#                 source_down, target_down, voxel_size)  # Assuming this function calculates the combined ratio
#                 update_metrics(combined_ratio_dict, source_filename_pcd, target_filename_pcd, combined_ratio) 
#             # o3d.visualization.draw_geometries([source_down, target_down])                           
#         print(index,'# ',source_filename_stl,'done')
#         index = index +1
#     # Rank the results based on the new combined ratios
#     combined_ratio_ranking = rank_metrics(combined_ratio_dict, lower_better=True)
#     return combined_ratio_ranking

# def calculate_and_visualize_subtracted_point_clouds(source, target, voxel_distance):
#     # Build KD-Trees for both point clouds
#     source_kd_tree = o3d.geometry.KDTreeFlann(source)
#     target_kd_tree = o3d.geometry.KDTreeFlann(target)
    
#     # Find source points close to any target point
#     source_close_indices = set()
#     for i, point in enumerate(source.points):
#         _, idx, _ = target_kd_tree.search_radius_vector_3d(point, voxel_distance)
#         if len(idx) > 0:
#             source_close_indices.add(i)
    
#     # Find target points close to any source point
#     target_close_indices = set()
#     for i, point in enumerate(target.points):
#         _, idx, _ = source_kd_tree.search_radius_vector_3d(point, voxel_distance)
#         if len(idx) > 0:
#             target_close_indices.add(i)
    
#     # Create new point clouds for the remaining points
#     remaining_source_points = [source.points[i] for i in range(len(source.points)) if i not in source_close_indices]
#     remaining_target_points = [target.points[i] for i in range(len(target.points)) if i not in target_close_indices]
    
#     remaining_source = o3d.geometry.PointCloud()
#     remaining_source.points = o3d.utility.Vector3dVector(remaining_source_points)
#     remaining_target = o3d.geometry.PointCloud()
#     remaining_target.points = o3d.utility.Vector3dVector(remaining_target_points)
    
#     # Visualize
#     remaining_source.paint_uniform_color([1, 0, 0])  # Red for source
#     remaining_target.paint_uniform_color([0, 1, 0])  # Green for target
#     o3d.visualization.draw_geometries([remaining_source, remaining_target])
    
#     # Calculate the ratio of remaining points
#     total_points = len(source.points) + len(target.points)
#     total_remaining_points = len(remaining_source_points) + len(remaining_target_points)
#     combined_ratio = total_remaining_points / total_points
    
#     return combined_ratio

In [7]:
import open3d as o3d
import numpy as np
import os
import copy
import pandas as pd

In [13]:
def calculate_aspect_ratio(pcd):
    obb = pcd.get_oriented_bounding_box()
    extents = np.sort(obb.extent)  # Sort extents to ensure consistent order
    aspect_ratio = extents[-1] / extents[0]  # Longest dimension over shortest
    return aspect_ratio

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])



def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=50))
    return pcd_down, pcd_fpfh

def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, aspect_ratio_threshold):
    aspect_ratio_source = calculate_aspect_ratio(source_down)
    aspect_ratio_target = calculate_aspect_ratio(target_down)
    
    # if aspect_ratio_source > aspect_ratio_threshold or aspect_ratio_target > aspect_ratio_threshold:
    #     # print("Skipping RANSAC due to high aspect ratio")
    #     return None  # Skipping RANSAC, might need additional handling based on your workflow
    
    distance_threshold = voxel_size * 1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(distance_threshold),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(50000, 0.999))
    return result

def refine_registration_with_icp(source, target, voxel_size):
    distance_threshold = voxel_size * 5
    icp_criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=500)
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), icp_criteria)
    return icp_result

def calculate_and_visualize_subtracted_point_clouds(source, target, voxel_distance):
    # Build KD-Trees for both point clouds
    source_kd_tree = o3d.geometry.KDTreeFlann(source)
    target_kd_tree = o3d.geometry.KDTreeFlann(target)
    
    # Find source points close to any target point
    source_close_indices = set()
    for i, point in enumerate(source.points):
        _, idx, _ = target_kd_tree.search_radius_vector_3d(point, voxel_distance)
        if len(idx) > 0:
            source_close_indices.add(i)
    
    # Find target points close to any source point
    target_close_indices = set()
    for i, point in enumerate(target.points):
        _, idx, _ = source_kd_tree.search_radius_vector_3d(point, voxel_distance)
        if len(idx) > 0:
            target_close_indices.add(i)
    
    # Create new point clouds for the remaining points
    remaining_source_points = [source.points[i] for i in range(len(source.points)) if i not in source_close_indices]
    remaining_target_points = [target.points[i] for i in range(len(target.points)) if i not in target_close_indices]

    
    # Calculate the ratio of remaining points
    total_points = len(source.points) + len(target.points)
    total_remaining_points = len(remaining_source_points) + len(remaining_target_points)
    combined_ratio = total_remaining_points / total_points
    
    return combined_ratio


def prepare_dataset(voxel_size, pcds_o3d):
    source = pcds_o3d[0]
    target = pcds_o3d[1]
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def scale_point_cloud(pcd, scale_factors):
    """
    Scale the point cloud by the given scale factors.
    This is a placeholder function. You should replace it with actual scaling logic.
    """
    points = np.asarray(pcd.points)
    scaled_points = points * scale_factors  # Apply scaling
    pcd_scaled = o3d.geometry.PointCloud()
    pcd_scaled.points = o3d.utility.Vector3dVector(scaled_points)
    return pcd_scaled


def get_centroid(pcd):
    """Compute the centroid of a point cloud."""
    return np.asarray(pcd.points).mean(axis=0)

def translate_point_cloud(pcd, translation):
    """Translate a point cloud by a given translation vector."""
    translated_pcd = pcd.translate(translation, relative=True)
    return translated_pcd
# Load all .pcd files into a dictionary for quick access
def load_all_pcds(pcd_file_paths):
    pcds_o3d = {}
    for pcd_path in pcd_file_paths:
        filename = os.path.basename(pcd_path)
        pcds_o3d[filename] = o3d.io.read_point_cloud(pcd_path)
    return pcds_o3d


In [15]:

def update_metrics(metrics_dict, source_filename, target_filename, value):
    if source_filename not in metrics_dict:
        metrics_dict[source_filename] = []
    metrics_dict[source_filename].append((target_filename, value))

def rank_metrics(metrics_dict, lower_better=True):
    ranking_dict = {}
    for key, values in metrics_dict.items():
        ranking_dict[key] = sorted(values, key=lambda x: x[1], reverse=not lower_better)
    return ranking_dict

def scale_align_pcds(pcds_o3d, cube_size):
    pcds_downsampled_scaled_and_normals = []
    centroids = []

    for pcd in pcds_o3d:
        obb = pcd.get_oriented_bounding_box()
        centroid = obb.center
        centroids.append(centroid)
    reference_centroid = np.mean(centroids, axis=0)

    for pcd in pcds_o3d:
        obb = pcd.get_oriented_bounding_box()
        extents = obb.extent
        scale_factors = np.full(3, cube_size / np.max(extents))
        centroid = obb.center
        scaled_points = ((np.asarray(pcd.points) - centroid) * scale_factors) + centroid
        translation = reference_centroid - centroid
        translated_and_scaled_points = scaled_points + translation
        pcd_scaled = o3d.geometry.PointCloud()
        pcd_scaled.points = o3d.utility.Vector3dVector(translated_and_scaled_points)
        pcds_downsampled_scaled_and_normals.append(pcd_scaled)

    return pcds_downsampled_scaled_and_normals

def main_process(ranking_dict, distance_dict, voxel_size, aspect_ratio_threshold, cube_size, pcds_down_mapped, df_parts):
    combined_ratio_dict = {}
    index = 1
    # Convert the "part_id_stl_file" column to a set for faster lookup
    valid_parts_set = set(df_parts['part_id_stl_file'])

    # Iterate over each part in ranking_dict
    for source_filename_stl, similar_parts_stl in ranking_dict.items():
        # Check if the source part is in the DataFrame's "part_id_stl_file" column
        if source_filename_stl not in valid_parts_set:
            continue  # Skip this source part if it's not in the DataFrame

        # Replace the .stl extension with .pcd for the source filename
        source_filename_pcd = source_filename_stl.replace('.stl', '.pcd')
        source_pcd = pcds_down_mapped[source_filename_pcd]

        # aspect_ratio_source = calculate_aspect_ratio(source_pcd)
        # if aspect_ratio_source > aspect_ratio_threshold:
        #     # Fetch top 50 tuples from the distance_dict for the given source file
        #     if source_filename_stl in distance_dict:
        #         top_similar_parts_by_distance = distance_dict[source_filename_stl][:50]
        #         # Update combined_ratio_dict with these parts
        #         combined_ratio_dict[source_filename_pcd] = top_similar_parts_by_distance
        #     continue
        top_similar_parts_stl = similar_parts_stl[:50]  # Get top 50 similar parts
        for target_filename_stl in top_similar_parts_stl:
            target_filename_pcd = target_filename_stl.replace('.stl', '.pcd')
            target_pcd = pcds_down_mapped[target_filename_pcd]
            # aspect_ratio_target = calculate_aspect_ratio(target_pcd)
            # if aspect_ratio_target > aspect_ratio_threshold:
            #     update_metrics(combined_ratio_dict, source_filename_pcd, target_filename_pcd, 1)
            #     continue                
            # Get deep copies for comparison and possible cropping
            source_pcd_copy = copy.deepcopy(source_pcd)
            target_pcd_copy = copy.deepcopy(target_pcd)

            # Crop dimension if needed
            
            source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
                voxel_size, [source_pcd_copy, target_pcd_copy])
            
            result_ransac = execute_global_registration(
                source_down, target_down, source_fpfh, target_fpfh, voxel_size, aspect_ratio_threshold)
            
            source_down.transform(result_ransac.transformation)
            result_icp = refine_registration_with_icp(source_down, target_down, voxel_size)
            source_down.transform(result_icp.transformation)
            combined_ratio = calculate_and_visualize_subtracted_point_clouds(
            source_down, target_down, voxel_size)  # Assuming this function calculates the combined ratio
            update_metrics(combined_ratio_dict, source_filename_pcd, target_filename_pcd, combined_ratio)

            # o3d.visualization.draw_geometries([source_down, target_down])                           
        print(index, '# ', source_filename_stl, 'done')
        index = index + 1
    # Rank the results based on the new combined ratios
    combined_ratio_ranking = rank_metrics(combined_ratio_dict, lower_better=True)
    return combined_ratio_ranking

    
#icp final filtered (with long parts dealt with)

# def main_process(ranking_dict, distance_dict, voxel_size, aspect_ratio_threshold, cube_size, pcds_down_mapped, df_parts):
#     combined_ratio_dict = {}
#     index = 1
#     # Convert the "part_id_stl_file" column to a set for faster lookup
#     valid_parts_set = set(df_parts['part_id_stl_file'])
    
#     # Iterate over each part in ranking_dict
#     for source_filename_stl, similar_parts_stl in ranking_dict.items():
#         # Check if the source part is in the DataFrame's "part_id_stl_file" column
#         if source_filename_stl not in valid_parts_set:
#             continue  # Skip this source part if it's not in the DataFrame

#         # Replace the .stl extension with .pcd for the source filename
#         source_filename_pcd = source_filename_stl.replace('.stl', '.pcd')
#         source_pcd = pcds_down_mapped[source_filename_pcd]

#         aspect_ratio_source = calculate_aspect_ratio(source_pcd)
#         if aspect_ratio_source > aspect_ratio_threshold:
#             # Fetch top 50 tuples from the distance_dict for the given source file
#             if source_filename_stl in distance_dict:
#                 top_similar_parts_by_distance = distance_dict[source_filename_stl][:50]
#                 # Update combined_ratio_dict with these parts
#                 combined_ratio_dict[source_filename_pcd] = top_similar_parts_by_distance
#             continue
#         top_similar_parts_stl = similar_parts_stl[:50]  # Get top 50 similar parts
#         for target_filename_stl in top_similar_parts_stl:
#             target_filename_pcd = target_filename_stl.replace('.stl', '.pcd')
#             target_pcd = pcds_down_mapped[target_filename_pcd]
#             aspect_ratio_target = calculate_aspect_ratio(target_pcd)
#             if aspect_ratio_target > aspect_ratio_threshold:
#                 update_metrics(combined_ratio_dict, source_filename_pcd, target_filename_pcd, 1)
#                 continue                
#             # Get deep copies for comparison and possible cropping
#             source_pcd_copy = copy.deepcopy(source_pcd)
#             target_pcd_copy = copy.deepcopy(target_pcd)

#             # Crop dimension if needed
            
#             source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
#                 voxel_size, [source_pcd_copy, target_pcd_copy])
            
#             result_ransac = execute_global_registration(
#                 source_down, target_down, source_fpfh, target_fpfh, voxel_size, aspect_ratio_threshold)
            
#             # Assuming result_ransac and result_icp operations are successful
#             if result_ransac:
#                 source_down.transform(result_ransac.transformation)
#                 result_icp = refine_registration_with_icp(source_down, target_down, voxel_size)
#                 source_down.transform(result_icp.transformation)
#                 combined_ratio = calculate_and_visualize_subtracted_point_clouds(
#                 source_down, target_down, voxel_size)  # Assuming this function calculates the combined ratio
#                 update_metrics(combined_ratio_dict, source_filename_pcd, target_filename_pcd, combined_ratio) 
#             # o3d.visualization.draw_geometries([source_down, target_down])                           
#         print(index, '# ', source_filename_stl, 'done')
#         index = index + 1
#     # Rank the results based on the new combined ratios
#     combined_ratio_ranking = rank_metrics(combined_ratio_dict, lower_better=True)
#     return combined_ratio_ranking

In [16]:
import pickle

with open('ranking_dict.pkl', 'rb') as file:
    ranking_dict = pickle.load( file)

with open('distance_dict.pkl', 'rb') as file:
    distance_dict = pickle.load( file)

with open('scores_dict.pkl', 'rb') as file:
    scores_dict = pickle.load( file)

In [17]:
# Assuming the directories are already defined as in your code
# Assuming the directories are already defined as in your code
current_dir = os.getcwd()
ermetal_dir = os.path.join(current_dir, 'ermetal')
pcd_directory = os.path.join(current_dir, 'pcds_20k_random')

# Make sure the pcd directory exists, create if it does not
os.makedirs(pcd_directory, exist_ok=True)

stl_file_paths = [os.path.join(ermetal_dir, f) for f in os.listdir(ermetal_dir) if f.endswith('.stl')]
stl_file_names = [os.path.basename(f) for f in stl_file_paths]

obb_extents_dict = {}  # Dictionary to store the sorted extents
pcds_o3d = []
voxel_size = 0.06
aspect_ratio_threshold = 14   # Adjust based on your requirements
cube_size = 1.5

for i, stl_file in enumerate(stl_file_names):
    pcd_file_name = stl_file.replace('.stl', '.pcd')
    pcd_file_path = os.path.join(pcd_directory, pcd_file_name)

    # Check if the .pcd file already exists
    if os.path.exists(pcd_file_path):
        # Read the existing point cloud
        point_cloud = o3d.io.read_point_cloud(pcd_file_path)
        # print(f"Read existing point cloud: {pcd_file_path}")
    else:
        # Process the mesh to generate a new point cloud
        mesh_path = os.path.join(ermetal_dir, stl_file)
        mesh = o3d.io.read_triangle_mesh(mesh_path)

        # Compute the OBB and get sorted extents
        obb = mesh.get_oriented_bounding_box()
        extents = sorted(obb.extent)  # Sort extents in ascending order
        obb_extents_dict[stl_file] = extents

        point_cloud = mesh.sample_points_uniformly(number_of_points=20_000)
        o3d.io.write_point_cloud(pcd_file_path, point_cloud)
        print(f"Generated and saved point cloud: {pcd_file_path}")

    pcds_o3d.append(point_cloud)

# Normalize and estimate normals for the point clouds
pcds_down = scale_align_pcds(pcds_o3d, cube_size)


In [18]:
voxel_size = 0.06
aspect_ratio_threshold = 14   # Adjust based on your requirements
cube_size = 1.5
# Assuming stl_file_names and pcds_down are already defined and correspond to each other
pcds_mapped = {stl_file_name.replace('.stl', '.pcd'): pcd for stl_file_name, pcd in zip(stl_file_names, pcds_down)}
curr=os.getcwd()
df_parts = pd.read_excel(curr+'\\labelled_files.xlsx')
# Call the main process function
combined_ratio_ranking = main_process(ranking_dict,distance_dict, voxel_size, aspect_ratio_threshold, cube_size, pcds_mapped,df_parts)

1 #  003_2021_Teklifid_2318.stl done
2 #  004_2020_Teklifid_342.stl done
3 #  007_2020_Teklifid_142.stl done
4 #  007_2021_Teklifid_2365.stl done
5 #  017_2020_Teklifid_286.stl done
6 #  024_2021_Teklifid_2397.stl done
7 #  029_2020_Teklifid_380.stl done
8 #  036_2020_Teklifid_404.stl done
9 #  037_2021_Teklifid_2457.stl done
10 #  040_2020_Teklifid_441.stl done
11 #  053_2022_Teklifid_3974.stl done
12 #  053_2022_Teklifid_3982.stl done
13 #  063_2020_Teklifid_732.stl done
14 #  064_2022_Teklifid_4015.stl done
15 #  070_2022_Teklifid_4075.stl done
16 #  073_2021_Teklifid_2570.stl done
17 #  073_2021_Teklifid_2574.stl done
18 #  093_2020_Teklifid_804.stl done
19 #  098_2022_Teklifid_4112.stl done
20 #  098_2022_Teklifid_4135.stl done
21 #  099_2022_Teklifid_4141.stl done
22 #  106_2020_Teklifid_918.stl done
23 #  114_2020_Teklifid_1047.stl done
24 #  114_2020_Teklifid_933.stl done
25 #  114_2020_Teklifid_935.stl done
26 #  120_2020_Teklifid_994.stl done
27 #  121_2020_Teklifid_1010.stl 

In [ ]:
len(combined_ratio_ranking)

In [19]:
# import pickle
# with open('icp_filtered_only.pkl','wb') as file:
#     pickle.dump(combined_ratio_ranking, file)

In [20]:
import pickle
with open('icp_filtered_final.pkl','rb') as file:
    combined_ratio_ranking =  pickle.load( file)

In [21]:
# # Convert .pcd extensions to .stl for both keys and values
combined_ratio_ranking_stl = {
    key.replace('.pcd', '.stl'): [(filename.replace('.pcd', '.stl'), score) for filename, score in value]
    for key, value in combined_ratio_ranking.items()
}


# scores_dict = {   
#     key: [scores for _, scores in value]
#     for key, value in combined_ratio_ranking_stl.items()
# }
# # # Create a new dictionary with .stl filenames without the scores
# ranking_dict = {
#     key: [filename for filename, _ in value]
#     for key, value in combined_ratio_ranking_stl.items()
# }

# Print or return ranking_dict as needed
# print(ranking_dict)

In [22]:
scores_dict = {   
    key: [scores for _, scores in value]
    for key, value in combined_ratio_ranking_stl.items()
}

In [23]:
ranking_dict = {
    key: [filename for filename, _ in value]
    for key, value in combined_ratio_ranking_stl.items()
}


In [ ]:
df['part_id_stl_file']

In [24]:
a = ''
# Read the Excel file
curr=os.getcwd()

df = pd.read_excel(curr+'\\labelled_files.xlsx')

# Loop through the dataframe
for index, row in df.iterrows():
    stl_file_name = row['part_id_stl_file']
    top_choice = row['top_choice_stl_file']

    # Check if stl_file_name is in ranking_dict
    if stl_file_name in ranking_dict:
        ranking = ranking_dict[stl_file_name]
        
        # Check if the top choice matches the first rank in ranking_dict
        if ranking[0] == top_choice:
            # print(f"{stl_file_name}: matching file {top_choice} has rank 1")
            pass
        else:
            
            # Find the rank of the top choice in the ranking list
            if top_choice in ranking:
                rank = ranking.index(top_choice) + 1
                if rank <= 5:
                    # print(f"{stl_file_name}: matching file {top_choice} has rank {rank}")
                    pass
                else:
                    print(f"{stl_file_name}: matching file {top_choice} is beyond top 5, ranked at {rank}")
                    pass
            else:
                pass
                # print(f"{stl_file_name}: matching file {top_choice} is not ranked")
    else:
        print(f"{stl_file_name} is not in the ranking dictionary")

#ALIGNMENT AND VISUALIZATION FUNCTIONS 

def visualize_mesh_with_triangles(mesh):
    """Visualizes a mesh with colored triangles and black lines along the edges of the triangles."""

    # mesh = translate_mesh_to_origin(mesh)
    vertex_colors = np.full((len(np.asarray(mesh.vertices)), 3), [0.8, 0.8, 0.8])
    mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
    lines = [[triangle[0], triangle[1]] for triangle in np.asarray(mesh.triangles)] + \
            [[triangle[1], triangle[2]] for triangle in np.asarray(mesh.triangles)] + \
            [[triangle[2], triangle[0]] for triangle in np.asarray(mesh.triangles)]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(mesh.vertices)),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.paint_uniform_color([0, 0, 0])
    return [mesh, line_set]

def visualize_meshes(filenames, directory_path):
    geometries_to_show = []
    current_x_position = 50
    spacing_factor = 3.5  # Adjust this factor to control spacing between the meshes

    for i, filename in enumerate(filenames):
        full_path = os.path.join(directory_path, filename)
        mesh_open3d = o3d.io.read_triangle_mesh(full_path)
        # mesh_open3d = align_mesh(mesh_open3d)
        # Compute the axis-aligned bounding box to get the x dimension
        aabb = mesh_open3d.get_axis_aligned_bounding_box()
        x_dim = aabb.get_extent()[0]
        # For the first mesh (target mesh), we start at (0,0,0) and account for its size in spacing
        if i == 0:
            mesh_open3d.translate(-aabb.get_center())
        else:
            mesh_open3d.translate((current_x_position, 0, 0) - aabb.get_center())
        geometries_to_show.extend(visualize_mesh_with_triangles(mesh_open3d))
        current_x_position += x_dim * spacing_factor  # Update the x position
    
    o3d.visualization.draw_geometries(geometries_to_show)

#ALIGNMENT AND VISUALIZATION FUNCTIONS 

#VISUALIZE BAD EXAMPLES, THE FIRST ONE IS THE TARGET PART, SECOND ONE IS THE ERMETAL TOP CHOICE, AND THE REST ARE ALGORITHM CHOICES

current_dir = os.getcwd()

# loading ermetal stl meshes
# stl_directory = os.path.join(current_dir,'updated_stl')
stl_directory = os.path.join(current_dir,'ermetal')

pcd_directory = os.path.join(current_dir,'pcd_5k')
# Initialize a list to store filenames for visualization along with their top choices
files_to_visualize_with_top_choice = []

# Loop through the dataframe
for index, row in df.iterrows():
    stl_file_name = row['part_id_stl_file']
    top_choice = row['top_choice_stl_file']

    if stl_file_name in ranking_dict:
        ranking = ranking_dict[stl_file_name]

        if ranking[0] != top_choice and top_choice in ranking:
            rank = ranking.index(top_choice) + 1
            if rank > 5:
                # Add the STL file, its top choice, and its top 5 similar parts to the visualization list
                files_to_visualize_with_top_choice.append((stl_file_name, top_choice, ranking[:3]))
    else:
        print(f"{stl_file_name} is not in the ranking dictionary")

# Visualize the collected STL files, their top choices, and their top 5 similar parts
directory_path = stl_directory  # Replace with the actual path to your STL files
for stl_file, top_choice, top_matches in files_to_visualize_with_top_choice:
    print('target file',stl_file,': ',combined_ratio_ranking_stl[stl_file],"\n")
    # print('ermetal top choice', top_choice, ": ", scores_dict[top_choice],'\n')

    # print('target file',stl_file,': ',normalized_dict[stl_file],"\n")
    # print('ermetal top choice', top_choice, ": ", normalized_dict[top_choice],'\n')
    
    # for i in range(len(top_matches)):
        # print('algorithm top match #',i+1, top_matches[i],": ",combined_ratio_ranking_stl[top_matches[i]],'\n')
        # print('algorithm top match #',i+1, top_matches[i],": ",normalized_dict[top_matches[i]],'\n')
        
    visualize_filenames = [stl_file, top_choice] + top_matches
    visualize_meshes(visualize_filenames, directory_path)



# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 7
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 34
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 12
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 19
# 543_2021_Teklifid_3311.stl: matching file 543_2021_Teklifid_3312.stl is beyond top 5, ranked at 12
# 997_2021_Teklifid_3926.stl: matching file 997_2021_Teklifid_3931.stl is beyond top 5, ranked at 13
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 45
# 098_2022_Teklifid_4112.stl: matching file 098_2022_Teklifid_4113.stl is beyond top 5, ranked at 33
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 27

# 036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 28
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 7
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 43
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 46
# 543_2021_Teklifid_3276.stl: matching file 543_2021_Teklifid_3277.stl is beyond top 5, ranked at 7
# 098_2022_Teklifid_4112.stl: matching file 098_2022_Teklifid_4113.stl is beyond top 5, ranked at 31
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 25

# 121_2020_Teklifid_1034.stl: matching file 121_2020_Teklifid_1035.stl is beyond top 5, ranked at 6
# 228_2020_Teklifid_1921.stl: matching file 228_2020_Teklifid_1922.stl is beyond top 5, ranked at 7
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 11
# 997_2021_Teklifid_3882.stl: matching file 997_2021_Teklifid_3915.stl is beyond top 5, ranked at 21
# 228_2020_Teklifid_1921.stl: matching file 228_2020_Teklifid_1922.stl is beyond top 5, ranked at 7
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 50
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 11
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 23
# 098_2022_Teklifid_4112.stl: matching file 098_2022_Teklifid_4113.stl is beyond top 5, ranked at 10
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 29

543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 49
098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 11
543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 24
543_2021_Teklifid_3276.stl: matching file 543_2021_Teklifid_3277.stl is beyond top 5, ranked at 10
098_2022_Teklifid_4112.stl: matching file 098_2022_Teklifid_4113.stl is beyond top 5, ranked at 11
140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 35
target file 543_2021_Teklifid_3322.stl :  [('079_2020_Teklifid_741.stl', 62.62362838426717), ('121_2020_Teklifid_1073.stl', 71.43501690954722), ('543_2021_Teklifid_3271.stl', 74.49047734282445), ('997_2021_Teklifid_3903.stl', 77.43767870605812), ('106_2021_Teklifid_2612.stl', 86.77838930986992), ('007_2021_Teklifid_2335.stl', 86.99158667543169), ('067_2022_Teklifid_4074.stl', 91.51234950741727), ('017_2

KeyboardInterrupt: 

In [ ]:
import pandas as pd

In [ ]:
distance_dict['036_2020_Teklifid_404.stl']